# Job Board Scraping Lab

In this lab you will first see a minimal but fully functional code snippet to scrape the LinkedIn Job Search webpage. You will then work on top of the example code and complete several chanllenges.

### Some Resources 

- [Requests library](http://docs.python-requests.org/en/master/#the-user-guide) documentation 
- [Beautiful Soup Doc](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Urllib](https://docs.python.org/3/library/urllib.html#module-urllib)
- [re lib](https://docs.python.org/3/library/re.html)
- [Scrapy](https://scrapy.org/)
- [List of HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes)
- [HTML basics](http://www.simplehtmlguide.com/cheatsheet.php)
- [CSS basics](https://www.cssbasics.com/#page_start)

In [3]:
# Import the required libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests

"""
This function searches job posts from LinkedIn and converts the results into a dataframe.
"""
def scrape_linkedin_job_search(keywords):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    # Assemble the full url with parameters
    scrape_url = ''.join([BASE_URL, 'keywords=', keywords])

    # Create a request to get the data from the server 
    page = requests.get(scrape_url)
    soup = BeautifulSoup(page.text, 'html.parser')

    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)

    # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
    # Then in each job card, extract the job title, company, and location data.
    titles = []
    companies = []
    locations = []
    for card in soup.select("div.result-card__contents"):
        title = card.findChild("h3", recursive=False)
        company = card.findChild("h4", recursive=False)
        location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
        titles.append(title.string)
        companies.append(company.string)
        locations.append(location.string)
    
    # Inject job titles, companies, and locations into the empty dataframe
    zipped = zip(titles, companies, locations)
    for z in list(zipped):
        data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)
    
    # Return dataframe
    return data

In [48]:
# Example to call the function

results = scrape_linkedin_job_search('data%20analysis')
results

,Title,Company,Location
0,Sr. Data Engineer,Cypress HCM,Miami/Fort Lauderdale Area
1,Data Architect,SessionM,Greater Boston Area
2,Settlement Analyst,Interactive Resources - iR,"Bellevue, Washington"
3,Senior Data Engineer,"Dynamo, LLC.",Greater Omaha Area
4,Sr. Statistician,Legend Biotech,"Piscataway, New Jersey"
5,Lead Strategic Data Analyst (SAS & Tableau) Lo...,Compass Technology Group,"Dallas, Texas"
6,IT Data Analyst,Navigant,"Gardena, California"
7,Senior Data Analyst,Redfish Technology,Greater Los Angeles Area
8,Cloud Data Engineer,Idexcel,"Merrimack, New Hampshire"
9,IT Senior Associate,Kearney & Company,"McLean, VA, US"


## Challenge 1

The first challenge for you is to update the `scrape_linkedin_job_search` function by adding a new parameter called `num_pages`. This will allow you to search more than 25 jobs with this function. Suggested steps:

1. Go to https://www.linkedin.com/jobs/search/?keywords=data%20analysis in your browser.
1. Scroll down the left panel and click the page 2 link. Look at how the URL changes and identify the page offset parameter.
1. Add `num_pages` as a new param to the `scrape_linkedin_job_search` function. Update the function code so that it uses a "for" loop to retrieve several pages of search results.
1. Test your new function by scraping 5 pages of the search results.

Hint: Prepare for the case where there are less than 5 pages of search results. Your function should be robust enough to **not** trigger errors. Simply skip making additional searches and return all results if the search already reaches the end.

In [16]:
# your code here

def scrape_linkedin_job_search_pages(keywords,total_page_number):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    # Assemble the full url with parameters
    scrape_url = ''.join([BASE_URL, 'keywords=', keywords])
    
    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)

    # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
    # Then in each job card, extract the job title, company, and location data.
    
    
    # Add a new parameter num_pages
    for num in range(total_page_number):
        if num == 0:
            page = requests.get(scrape_url)
        else:
            page = requests.get(''.join([scrape_url,'&start=',str(total_page_number*25)]))
        soup = BeautifulSoup(page.text, 'html.parser')
        titles = []
        companies = []
        locations = []   
        
        for card in soup.select("div.result-card__contents"):
            title = card.findChild("h3", recursive=False)
            company = card.findChild("h4", recursive=False)
            location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
            titles.append(title.string)
            companies.append(company.string)
            locations.append(location.string)
        zipped = zip(titles, companies, locations)
        for z in list(zipped):
            data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)
    
    
    # Return dataframe
    return data

In [17]:
scrape_linkedin_job_search_pages('data%20analysis',3)

,Title,Company,Location
0,Compensation Analyst,"TalentBurst, an Inc 5000 company","San Francisco, California"
1,Cloud Data Engineer,Idexcel,"Merrimack, New Hampshire"
2,IT Senior Associate,Kearney & Company,"McLean, VA, US"
3,Business Intelligence Analyst,"CFO Alliance, Inc.","Tampa, Florida"
4,Intern: Data Analysis/Process Improvement Prog...,Juniper Networks,"Sunnyvale, CA, US"
...,...,...,...
70,Data Analyst,Brooksource,"Philadelphia, Pennsylvania"
71,Data Analyst,"Partner's Consulting, Inc.","Philadelphia, Pennsylvania"
72,Data Analyst,New York Life Insurance Company,"New York City, NY, US"
73,Data Analyst,24 Seven LLC,"Portland, Oregon"


## Challenge 2

Further improve your function so that it can search jobs in a specific country. Add the 3rd param to your function called `country`. The steps are identical to those in Challange 1.

In [34]:
def scrape_linkedin_job_search_country(keywords,total_page_number, country):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    # Assemble the full url with parameters
    if country == True:
        scrape_url = ''.join([BASE_URL, "keywords=", keywords.replace(" ","%20")])
    else:            
        scrape_url = ''.join([BASE_URL, "keywords=", keywords.replace(" ","%20"),"&location=",country.replace(" ","%20")])
    
    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)

    # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
    # Then in each job card, extract the job title, company, and location data.
    
    
    # Add a new parameter num_pages
    for num in range(total_page_number):
        if num == 0:
            page = requests.get(scrape_url)
        else:
            page = requests.get(''.join([scrape_url,'&start=',str(total_page_number*25)]))
        soup = BeautifulSoup(page.text, 'html.parser')
        titles = []
        companies = []
        locations = []   
        
        for card in soup.select("div.result-card__contents"):
            title = card.findChild("h3", recursive=False)
            company = card.findChild("h4", recursive=False)
            location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
            titles.append(title.string)
            companies.append(company.string)
            locations.append(location.string)
        zipped = zip(titles, companies, locations)
        for z in list(zipped):
            data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)
    
    
    # Return dataframe
    return data

In [38]:


scrape_linkedin_job_search_country('data analysis',3,"United Kingdom")

,Title,Company,Location
0,Senior Data Analyst - Programmatic,round8 talent search,"London, United Kingdom"
1,"Senior Manager, Evaluation Impact & Learning",GALVmed,"Edinburgh, Scotland, United Kingdom"
2,Data & Insights Manager - £45k,Moriati Digital Recruitment (UK) LTD,"London, United Kingdom"
3,BI Analyst,NCS - National Citizen Service Trust,"London, United Kingdom"
4,Data Analyst (People Analytics) – 6 mth FTC,Argus Media,"WC1X 8NL, Camden, England, United Kingdom"
...,...,...,...
70,Data Analyst,Attivo Group,"Jessop House Jessop Avenue, Cheltenham GL50 3S..."
71,Business Analyst - Finance Operations Program ...,Amazon,"London, GB"
72,FP&A Manager,PSAV,"Uxbridge, Greater London, United Kingdom"
73,Data Science Analysis - Analyst,BT,"Ipswich, GB"


## Challenge 3

Add the 4th param called `num_days` to your function to allow it to search jobs posted in the past X days. Note that in the LinkedIn job search the searched timespan is specified with the following param:

```
f_TPR=r259200
```

The number part in the param value is the number of seconds. 259,200 seconds equal to 3 days. You need to convert `num_days` to number of seconds and supply that info to LinkedIn job search.

In [43]:

def scrape_linkedin_job_search_days(keywords,total_page_number, country, days):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    # Assemble the full url with parameters
    if country == True:
        scrape_url = ''.join([BASE_URL,"f_TPR=r",str(days*86400),"&", "keywords=", keywords.replace(" ","%20")])
    else:            
        scrape_url = ''.join([BASE_URL,"f_TPR=r",str(days*86400),"&", "keywords=", keywords.replace(" ","%20"),"&location=",country.replace(" ","%20")])
    
    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)

    # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
    # Then in each job card, extract the job title, company, and location data.
    
    
    # Add a new parameter num_pages
    for num in range(total_page_number):
        if num == 0:
            page = requests.get(scrape_url)
        else:
            page = requests.get(''.join([scrape_url,'&start=',str(total_page_number*25)]))
        soup = BeautifulSoup(page.text, 'html.parser')
        titles = []
        companies = []
        locations = []
        
        for card in soup.select("div.result-card__contents"):
            title = card.findChild("h3", recursive=False)
            company = card.findChild("h4", recursive=False)
            location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
            seniority_level= card.findChild("")
            titles.append(title.string)
            companies.append(company.string)
            locations.append(location.string)
        zipped = zip(titles, companies, locations)
        for z in list(zipped):
            data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)
    
    
    # Return dataframe
    return data

In [46]:
pd.set_option("display.max_rows",100)

scrape_linkedin_job_search_days("data analysis", 3, "Germany", 3)

,Title,Company,Location
0,Professional Consultant (f/m/d) Procurement Da...,CAMELOT Management Consultants,"Mannheim, Baden-Württemberg, Germany"
1,Sales Operations Manager (m/f/d),Personio,"München, Bayern, Deutschland"
2,Product Owner (m/f/d),Limehome GmbH,"München, Bayern, Deutschland"
3,Data Architect - McKinsey Digital,McKinsey & Company,"Berlin, DE"
4,Professional Consultant (f/m/d) SAP Master Dat...,Camelot ITLab,"Mannheim, Baden-Württemberg, Germany"
5,Senior Data Analyst,Amazon,"Munich, DE"
6,Deep Learning Engineer- Deformable Object Trac...,Apple,"Munich, DE"
7,Microsoft Analytics Consultant (w/m/x),Avanade,"Düsseldorf, DE"
8,(Senior) CSE Power Analyst,Vattenfall,"Hamburg Area, Germany"
9,Data Analyst,Euro London Appointments,"Berlin, Berlin, Germany"


## Bonus Challenge

Allow your function to also retrieve the "Seniority Level" of each job searched. Note that the Seniority Level info is not in the initial search results. You need to make a separate search request for each job card based on the `currentJobId` value which you can extract from the job card HTML.

After you obtain the Seniority Level info, update the function and add it to a new column of the returned dataframe.

In [69]:
# your code here

TEST_URL = "https://www.linkedin.com/jobs/search/?keywords=data%20analysis"

In [70]:
import bs4

response= requests.get(TEST_URL).content
#html_bytes = response.content
soup = BeautifulSoup(response, "html.parser")
soup

<!DOCTYPE html>
<html lang="en"><head><meta content="d_jobs_guest_search" name="pageKey"/><meta content="en_US" name="locale"/><meta data-app-id="com.linkedin.jobs-guest-frontend.d_web" data-custom-tracking-code="" data-tracking-page-type="" id="config"><link href="https://www.linkedin.com/jobs/data-analysis-jobs" rel="canonical"/><link href="https://static-exp1.licdn.com/scds/common/u/images/logos/favicons/v1/favicon.ico" rel="icon"/><script>function getDfd() {let yFn,nFn;const p=new Promise((y, n)=>{yFn=y;nFn=n;});p.resolve=yFn;p.reject=nFn;return p;}window.lazyloader = getDfd();window.tracking = getDfd();window.impressionTracking = getDfd();</script><script async="" src="https://static-exp1.licdn.com/sc/h/gyr9f1lbpyirhri42ukub2rz"></script><meta content="Today's top 190,000+ Data Analysis jobs in United States. Leverage your professional network, and get hired. New Data Analysis jobs added daily." name="description"/><meta content="noarchive" name="robots"/><meta content="width=devi

In [63]:
seniority= soup.body.find("p",class_="jobs-box__body js-formatted-exp-body")

type(seniority)

NoneType

In [65]:
seniority= soup.body.find_all("label", class_="filter-list__list-item-label")
seniority

[<label class="filter-list__list-item-label" for="TIME_POSTED-0">Past 24 hours <span class="filter-list__label-count">(6,132)</span></label>,
 <label class="filter-list__list-item-label" for="TIME_POSTED-1">Past Week <span class="filter-list__label-count">(45,912)</span></label>,
 <label class="filter-list__list-item-label" for="TIME_POSTED-2">Past Month <span class="filter-list__label-count">(125,492)</span></label>,
 <label class="filter-list__list-item-label" for="TIME_POSTED-3">Any Time <span class="filter-list__label-count">(194,929)</span></label>,
 <label class="filter-list__list-item-label" for="EXPERIENCE-0">Associate <span class="filter-list__label-count">(69,473)</span></label>,
 <label class="filter-list__list-item-label" for="EXPERIENCE-1">Entry level <span class="filter-list__label-count">(63,826)</span></label>,
 <label class="filter-list__list-item-label" for="EXPERIENCE-2">Mid-Senior level <span class="filter-list__label-count">(24,708)</span></label>,
 <label class="f

In [ ]:
<p class="jobs-box__body js-formatted-exp-body">Mid-Senior level</p>